In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

# Define parameters
img_width, img_height = 256, 256
batch_size = 32
epochs = 50
split_dir = 'Split_Data'

# Define data directories
train_data_dir = os.path.join(split_dir, 'train')
validation_data_dir = os.path.join(split_dir, 'val')
test_data_dir = os.path.join(split_dir, 'test')

# Create ImageDataGenerators for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


# Load the ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')  # 8 classes
])

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the model
model.save('resnet50_model.h5')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate predictions
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Print confusion matrix and classification report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

Found 480 images belonging to 8 classes.
Found 160 images belonging to 8 classes.
Found 160 images belonging to 8 classes.
Epoch 1/50
15/15 [==============================] - 42s 3s/step - loss: 3.0631 - accuracy: 0.1500 - val_loss: 2.1526 - val_accuracy: 0.1250
Epoch 2/50
15/15 [==============================] - 58s 4s/step - loss: 2.8300 - accuracy: 0.2062 - val_loss: 2.3759 - val_accuracy: 0.1250
Epoch 3/50
15/15 [==============================] - 53s 4s/step - loss: 2.4171 - accuracy: 0.2583 - val_loss: 2.5621 - val_accuracy: 0.1250
Epoch 4/50
15/15 [==============================] - 52s 4s/step - loss: 2.2219 - accuracy: 0.3104 - val_loss: 2.5159 - val_accuracy: 0.1250
Epoch 5/50
15/15 [==============================] - 37s 2s/step - loss: 2.2280 - accuracy: 0.2708 - val_loss: 2.5598 - val_accuracy: 0.1250
Epoch 6/50
15/15 [==============================] - 53s 4s/step - loss: 2.1825 - accuracy: 0.2854 - val_loss: 2.7061 - val_accuracy: 0.1250
Epoch 7/50
15/15 [===================

/opt/conda/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 7s 1s/step - loss: 2.1983 - accuracy: 0.3625
Test Accuracy: 36.25%
5/5 [==============================] - 7s 1s/step
Confusion Matrix
[[ 0  0  1  2  0  5  7  5]
 [ 1  1  1  2  0  3  5  7]
 [ 0  0  0  0  1  3  9  7]
 [ 0  0  2  2  0  4  9  3]
 [ 0  0  0  1  1  3  8  7]
 [ 0  0  2  4  1  5  2  6]
 [ 0  1  0  1  0  5  8  5]
 [ 0  0  0  0  0  3 11  6]]
Classification Report
              precision    recall  f1-score   support

         bus       0.00      0.00      0.00        20
         car       0.50      0.05      0.09        20
     minivan       0.00      0.00      0.00        20
   motorbike       0.17      0.10      0.12        20
      pickup       0.33      0.05      0.09        20
     scooter       0.16      0.25      0.20        20
       truck       0.14      0.40      0.20        20
         van       0.13      0.30      0.18        20

    accuracy                           0.14       160
   macro avg       0.18      0.14      0.11   

In [6]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate predictions
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)

# Get true labels
y_true = test_generator.classes

# Print confusion matrix and classification report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

5/5 [==============================] - 6s 1s/step - loss: 1.6282 - accuracy: 0.3875
Test Accuracy: 38.75%
5/5 [==============================] - 6s 1s/step
Confusion Matrix
[[3 3 1 3 0 8 1 1]
 [4 4 3 2 1 4 0 2]
 [2 2 1 5 3 4 2 1]
 [2 1 2 2 1 3 1 8]
 [1 4 2 2 4 3 1 3]
 [2 4 2 4 0 3 3 2]
 [3 2 3 0 5 4 1 2]
 [1 2 1 5 1 5 1 4]]
Classification Report
              precision    recall  f1-score   support

         bus       0.17      0.15      0.16        20
         car       0.18      0.20      0.19        20
     minivan       0.07      0.05      0.06        20
   motorbike       0.09      0.10      0.09        20
      pickup       0.27      0.20      0.23        20
     scooter       0.09      0.15      0.11        20
       truck       0.10      0.05      0.07        20
         van       0.17      0.20      0.19        20

    accuracy                           0.14       160
   macro avg       0.14      0.14      0.14       160
weighted avg       0.14      0.14      0.14       160

